In [ ]:
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from scipy.ndimage  import gaussian_filter1d

import json
import re
import os

#import copy

In [ ]:
from masking import *
from latencies import *
from excitation import *
from deconv import *
from ur import *
from tuning import *
from test import *
from ur import *

from suppression import *

from data import CAPData


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Simulate CAPs

In [ ]:
#masking degree function
#mdFunc=SigmoidMaskingDegreeFunction(30, 2*1/10., requires_grad=False)
mdFunc=WeibullCDF_IOFunc(0, 30, 7)
#latencies
lat0=Eggermont1976clickLatencies80dB #latencies

#Unitary response
#defined later
#ur0Wang=URWang1979m
#t=np.linspace(5e-4, 10e-3, num=500)
#u00=ur0Wang.u(t-2e-3) #shift
#sig=0.2e-3 #std of gaussian kernel in s
#sig2=sig/(t[1]-t[0])
#u0 = gaussian_filter1d(u00, sigma=sig2)
#u0/=np.amax(np.abs(u0))
#u0*=0.5*np.amax(np.abs(u00))

#tuning
#BW10_0=500
#BW10Func=constant_BW10(BW10_0, requires_grad=False)

BW10Func=Q10PowerLaw(2, 1000, 0.5, requires_grad=False)

plotMaskingDegreeFunc(mdFunc)
plotLatencies(lat0)

Suppression parameters

In [ ]:
np.log10(3.8)

In [ ]:
supp_freq_factor=1.14 #(Altoe et al 2021)

supp_BW10Func=Q10PowerLaw(0.52, 1000, 0.58, requires_grad=False)   #10kHz -> 3.8   np.log10(3.8)=0.57978
#Charaziak and Siegel 2014

suppFunc=LogLinearSuppression(0.2, 0.1)
suppAmount=SuppressionAmount(suppFunc, supp_BW10Func, supp_freq_factor)

In [ ]:
supp_BW10Func(5000)

Example: two maskers

In [ ]:
json_txt='''{
    "type":"noise-bands", 
    "n_bands":2, 
    "bands":[
        {
            "amplitude":0.05, 
            "fc_low":3500, 
            "fc_high":4100
        }, 
        {
            "amplitude":0.1, 
            "fc_low":5000, 
            "fc_high":12000
        }
    ], 
    "name":"2bands-6dB"
}'''

json_txt2='''{
    "type":"noise-bands", 
    "comment":"created with noiseBandMaskerDesigner", 
    "n_bands":1, 
    "bands":{
        "amplitude":0.1, 
        "fc_low":4800, 
        "fc_high":12000
    }, 
    "name":"4-hp-4800Hz"
}'''

json_txt3='''{
    "type":"noise-bands", 
    "n_bands":2, 
    "bands":[
        {
            "amplitude":0.1, 
            "fc_low":3500, 
            "fc_high":4100
        }, 
        {
            "amplitude":0.1, 
            "fc_low":5000, 
            "fc_high":12000
        }
    ], 
    "name":"2bands-0dB"
}'''


json_txt4='''{
    "type":"noise-bands", 
    "n_bands":2, 
    "bands":[
        {
            "amplitude":0.03, 
            "fc_low":3000, 
            "fc_high":4200
        }, 
        {
            "amplitude":0.1, 
            "fc_low":5000, 
            "fc_high":12000
        }
    ], 
    "name":"2bands-10dBbis"
}'''

json_txt_ref='''{
    "type":"noise-bands", 
    "n_bands":1, 
    "bands":[
        {
            "amplitude":0.1, 
            "fc_low":6000, 
            "fc_high":12000
        }
    ], 
    "name":"hp6000"
}'''


json_txt_ref='''{
    "type":"noise-bands", 
    "n_bands":1, 
    "bands":[
        {
            "amplitude":0.1, 
            "fc_low":4000, 
            "fc_high":12000
        }
    ], 
    "name":"hp4000"
}'''

#stim_struct=json.loads(json_txt)

#Alternative: load from json files
stim_folder='/home/fdeloche/Documents/Others/CAP-acquisition/stimuli/stim_files/test5kHz-difflevelsb/'  # b or not b?

#with open(f'{stim_folder}/4-hp6000-narrowband5kHz-32dB.json', 'r') as f:
#with open(f'{stim_folder}/14-hp6000-narrowband4500-20dB.json', 'r') as f:
#with open(f'{stim_folder}/18-hp6200-multnarrowband-attnnotch30dB.json', 'r') as f:
#with open(f'{stim_folder}/25-hp5400-narrowband4300-30dB.json', 'r') as f:
with open(f'{stim_folder}/15-hp6200-gradualamp.json', 'r') as f:
    json_txt=f.read()

#with open(f'{stim_folder}/6-hp6000-narrowband5kHz-26dB.json', 'r') as f:
#with open(f'{stim_folder}/13-hp6000-narrowband4000-20dB.json', 'r') as f:
#with open(f'{stim_folder}/19-hp6200-multnarrowband-attnnotch25dB.json', 'r') as f:
#with open(f'{stim_folder}/15-hp6200-gradualamp.json', 'r') as f:
with open(f'{stim_folder}/19-notch5300-bw1400.json', 'r') as f:
    json_txt2=f.read()
    

#with open(f'{stim_folder}/8-hp6000-narrowband5kHz-20dB.json', 'r') as f:
#with open(f'{stim_folder}/12-hp6000-narrowband4000-30dB.json', 'r') as f:
#with open(f'{stim_folder}/20-hp6200-multnarrowband-attnnotch35dB.json', 'r') as f:
#with open(f'{stim_folder}/27-hp6100-multbands.json', 'r') as f:
with open(f'{stim_folder}/14-hp6200-gradualamp.json', 'r') as f:
    json_txt3=f.read()


#with open(f'{stim_folder}/10-hp6000-narrowband5kHz-14dB.json', 'r') as f:
#with open(f'{stim_folder}/11-hp6000-narrowband4500-30dB.json', 'r') as f:
#with open(f'{stim_folder}/15-hp6000-narrowband4500-14dB.json', 'r') as f:
#with open(f'{stim_folder}/13-hp6200-gradualamp.json', 'r') as f:
#with open(f'{stim_folder}/26-hp6300-multbands.json', 'r') as f:   
with open(f'{stim_folder}/21-notch5300-bw1800-nonotch.json', 'r') as f:
    json_txt4=f.read()


maskingConditions=MaskingConditions.from_json_strings([json_txt, json_txt2, json_txt3, json_txt4])

#maskingConditionsRef=MaskingConditions.from_json_strings([json_txt_ref, json_txt_ref, json_txt_ref, json_txt_ref])
maskingConditionsRef=MaskingConditions.from_json_strings([ json_txt3,  json_txt4, json_txt,  json_txt4])


#maskingConditions.add_json_strings([json_txt])

In [ ]:
#maskingConditions.get_tensor_lists()
maskingConditions

In [ ]:
pl.figure(figsize=(11,10))
axlist=plotMaskingExcitations( BW10Func, maskingConditions)
plotMaskingExcitations( supp_BW10Func, maskingConditions, axlist=axlist, freq_factor=supp_freq_factor)
#pl.savefig('toymodel_maskers3.svg')

In [ ]:
maskingConditions.set_amp0_dB(60)
maskingConditionsRef.set_amp0_dB(60)

In [ ]:
pl.figure(figsize=(11,10))
axlist=plotMaskingAmountExcitations( BW10Func, maskingConditions, mdFunc)

plotMaskingAmountExcitations( BW10Func, maskingConditions, mdFunc, axlist=axlist, suppressionAmount=suppAmount)


### Interactive plots

In [ ]:
@interact_manual( BW0=widgets.IntSlider(value=500, min=100, max=2000, step=50),  BW1=widgets.IntSlider(value=500, min=100, max=2000, step=50), BWsupp=widgets.IntSlider(value=3000, min=1500, max=4500, step=50),
         a=(0.01, 0.6,0.02), freq_factor=(1.0, 2., 0.05), I0supp=(1, 100, 5), useRefMaskers=False)
def plotMaskingExcitations_interact(BW0, BW1, BWsupp, a, freq_factor, I0supp, useRefMaskers):
    #BW1=BW0*(1000/800)
    BW10func0=constant_BW10(BW0)
    
    BW10func1=constant_BW10(BW1)
    BW10supp=constant_BW10(BWsupp)
    
    
    
    pl.figure(figsize=(12, 10))
    

    suppFunc=LogLinearSuppression(a, I0supp)
    suppAmount=SuppressionAmount(suppFunc, BW10supp, freq_factor)
    
    refMaskers= maskingConditionsRef if useRefMaskers else None
    axlist=plotMaskingAmountExcitations( BW10func0, maskingConditions, mdFunc, suppressionAmount=suppAmount,
                refMaskers=refMaskers)


    plotMaskingAmountExcitations( BW10func1, maskingConditions, mdFunc, suppressionAmount=suppAmount, axlist=axlist,
                                refMaskers=refMaskers)


In [ ]:

@interact(I0=(30, 100), I1=(30,100), BW0=widgets.IntSlider(value=500, min=100, max=2500, step=50), BW1=widgets.IntSlider(value=500, min=100, max=2500, step=50))
def plotMaskingAmountExcitations_interact(I0, I1, BW0, BW1):
    BW10func0=constant_BW10(BW0)
    
    #BW1=BW0*(1000/800)
    BW10func1=constant_BW10(BW1)
    
    pl.figure(figsize=(12, 10))
    
    maskingConditions.set_amp0_dB(I0)
    
    axlist=plotMaskingAmountExcitations( BW10func0, maskingConditions, mdFunc)
    
    maskingConditions.set_amp0_dB(I1)
    axlist=plotMaskingAmountExcitations( BW10func1, maskingConditions, mdFunc, axlist=axlist)